In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import json

# These commands below set some options for pandas and to have matplotlib show the charts in the notebook
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.10f}'.format
%matplotlib inline

# Load the data
# We have this defaulted to the folder OUTSIDE of your repo - please change it as needed
population = pd.read_csv('Population by Age and Sex - US, States, Counties.csv')

def load_json(file_name: str):
    with open(file_name, 'r') as file:
        file_data = json.load(file)
    rows = []
    for person in file_data:
        base_info = {
            "bioguide_id": person["id"].get("bioguide"),
            "govtrack_id": person["id"].get("govtrack"),
            "icpsr_id": person["id"].get("icpsr"),
            "wikipedia": person["id"].get("wikipedia"),
            "wikidata": person["id"].get("wikidata"),
            "first_name": person["name"]["first"],
            "last_name": person["name"]["last"],
            "gender": person["bio"].get("gender"),
            "birthday": person["bio"].get("birthday"),
        }
        for term in person["terms"]:
            row = base_info.copy()
            row.update({
                "type": term["type"],
                "start": term["start"],
                "end": term["end"],
                "state": term["state"],
                "district": term.get("district"),
                "party": term.get("party"),
                "class": term.get("class"),
            })
            rows.append(row)

    # Convert to DataFrame
    legislators = pd.DataFrame(rows)
    return legislators

# load both historical & current into dataframes
incumbent = load_json('legislators-current.json')
non_incumbent = load_json('legislators-historical.json')

# filter historical from people born after 1900
non_incumbent['birthday'] = pd.to_datetime(non_incumbent['birthday'])
non_incumbent = non_incumbent[non_incumbent['birthday'].dt.year >= 1900] 

# check shape before
#print(incumbent.shape)
#print(non_incumbent.shape)

# merged dataframe: all legislators incumbent & non_incumbent 
all_legislators = pd.concat([incumbent, non_incumbent], ignore_index=True)

#check shape after
#print(all_legislators.columns)

# get earliest term start & latest term end and put in dataframe with bioguide as key
earliest_to_latest_terms = all_legislators.groupby('bioguide_id').agg(
    Earliest_Start_Date=('start', 'min'),
    Latest_End_Date=('end', 'max')
).reset_index()

# drop duplicates & term start/end now that we have the earliest term start & latest term end
all_legislators = all_legislators.drop_duplicates(subset='bioguide_id')
all_legislators['bioguide_id'].value_counts()
all_legislators = all_legislators.drop(['start', 'end'], axis=1)

# 
legislators_with_allterms = pd.merge(earliest_to_latest_terms, all_legislators, on='bioguide_id', how='left')
#print("\nInner Join:")
legislators_with_allterms.head()
#test = merged_inner[merged_inner['last_name'] == 'Sanders']
#test

# Convert dates to datetime
legislators_with_allterms['Earliest_Start_Date'] = pd.to_datetime(legislators_with_allterms['Earliest_Start_Date'])
legislators_with_allterms['Latest_End_Date'] = pd.to_datetime(legislators_with_allterms['Latest_End_Date'])

# Extract years from Start_Date and End_Date & filter End date to >= 2000
# legislators_with_allterms['Year_Start'] = legislators_with_allterms['Earliest_Start_Date'].dt.year
# legislators_with_allterms['Year_End'] = legislators_with_allterms['Latest_End_Date'].dt.year
legislators_with_allterms = legislators_with_allterms[legislators_with_allterms['Latest_End_Date'].dt.year >= 2000] 

print(legislators_with_allterms.head())

total_houserep = 435
total_senate = 100


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [2]:
##Gabby test cell
filtered_pop = population[population['Description'] == 'U.S.']

# Remove unnecessary columns
filtered_pop = filtered_pop.drop(['IBRC_Geo_ID', 'Statefips', 'Countyfips'], axis=1)

# Function to get percentages of populations
def percent(row, name):
    base = row['Total Population']
    return row[name]/base

for c in filtered_pop.columns:
    if 'population' in c.lower():
        filtered_pop['% ' + c] = filtered_pop.apply(lambda row: percent(row, c), axis = 1)
#filtered_pop['0-4 Percent'] = filtered_pop.apply(lambda row: percent(row, "Population 0-4"), axis = 1)
filtered_pop.columns
filtered_pop.head()

,Description,Year,Total Population,Population 0-4,Population 5-17,Population 18-24,Population 25-44,Population 45-64,Population 65+,Population Under 18,...,% Population 5-17,% Population 18-24,% Population 25-44,% Population 45-64,% Population 65+,% Population Under 18,% Population 18-54,% Population 55+,% Male Population,% Female Population
0,U.S.,2000,"282,162,411.0000000000","19,178,293.0000000000","53,197,896.0000000000","27,315,274.0000000000","84,973,340.0000000000","62,428,040.0000000000","35,069,568.0000000000","72,376,189.0000000000",...,0.1885364383,0.0968069202,0.3011504605,0.2212486056,0.1242885892,0.2565054245,0.5326279552,0.2108666204,0.4906514887,0.5093485113
1,U.S.,2001,"284,968,955.0000000000","19,298,217.0000000000","53,372,958.0000000000","27,992,652.0000000000","84,523,274.0000000000","64,491,563.0000000000","35,290,291.0000000000","72,671,175.0000000000",...,0.1872939387,0.0982305318,0.2966052004,0.2263108380,0.1238390722,0.2550143576,0.5330482192,0.2119374231,0.4909008141,0.5090991859
2,U.S.,2002,"287,625,193.0000000000","19,429,192.0000000000","53,507,265.0000000000","28,480,708.0000000000","83,990,295.0000000000","66,695,526.0000000000","35,522,207.0000000000","72,936,457.0000000000",...,0.1860312181,0.0990202134,0.2920129983,0.2318834637,0.1235017233,0.2535816012,0.5300759485,0.2163424502,0.4910229091,0.5089770909
3,U.S.,2003,"290,107,933.0000000000","19,592,446.0000000000","53,508,312.0000000000","28,916,746.0000000000","83,398,001.0000000000","68,828,899.0000000000","35,863,529.0000000000","73,100,758.0000000000",...,0.1844427743,0.0996758196,0.2874723216,0.2372527297,0.1236213317,0.2519777975,0.5278542349,0.2201679676,0.4909514039,0.5090485961
4,U.S.,2004,"292,805,298.0000000000","19,785,885.0000000000","53,511,850.0000000000","29,302,179.0000000000","83,066,831.0000000000","70,935,234.0000000000","36,203,319.0000000000","73,297,735.0000000000",...,0.1827557437,0.1000739372,0.2836930601,0.2422607599,0.1236429779,0.2503292649,0.5259431474,0.2237275877,0.4912069999,0.5087930001


In [3]:
##Ambro Test cell


In [4]:
##Uma test cell

In [5]:
##Chad test cell